In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from pandarallel import pandarallel

# Initialization
pandarallel.initialize(progress_bar=True)

df = pd.read_csv("emails_content.csv")
# check for null value
df.isnull().sum()
#Pre-processing
tokenizer = RegexpTokenizer("[A-Za-z]+")
df.content[0]
tokenizer.tokenize(df.content[0])

['Here', 'is', 'our', 'forecast']

In [3]:
df['text_tokenized'] = df.content.map(lambda t: tokenizer.tokenize(str(t)))
stemmer = SnowballStemmer("english")
df['text_stemmed'] = df['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])
df['text_sent'] = df['text_stemmed'].map(lambda l: ' '.join(l))

## Prova di Classificazione email

In [ ]:
from os import walk
from string import punctuation
from random import shuffle
from collections import Counter

# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
# Read the whole data from the Enron Dataset into a variable allData.

pathwalk = walk(r"enron-spam/")

allHamData, allSpamData = [], []
for root, dr, file in pathwalk:
    if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allHamData.append(" ".join(ip.readlines()))

    elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allSpamData.append(" ".join(ip.readlines()))
                # remove all redundant data

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))
# storing it in a dataframe

hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData,
                       "label": labels})

# get an overview of the data

raw_df.parallel_label.hist(bins=3)
raw_df.head(10)

In [ ]:
# check for null value
raw_df.isnull().sum()
#Pre-processing
tokenizer = RegexpTokenizer("[A-Za-z]+")
raw_df.email[0]
tokenizer.tokenize(raw_df.email[0])

In [ ]:
raw_df['text_tokenized'] = raw_df.email.map(lambda t: tokenizer.tokenize(str(t)))
stemmer = SnowballStemmer("english")
raw_df['text_stemmed'] = raw_df['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])
raw_df['text_sent'] = raw_df['text_stemmed'].map(lambda l: ' '.join(l))

#Visualizzation

In [ ]:
bad_emails = raw_df[raw_df.label == 'spam']
good_emails = raw_df[raw_df.label == 'ham']

#Creating Model

In [ ]:
cv = CountVectorizer()
feature = cv.fit_transform(raw_df.text_sent)
trainX, testX, trainY, testY = train_test_split(feature, raw_df.label)

#Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(trainX,trainY)
lr.score(testX,testY)
Scores_ml = {}
Scores_ml['Logistic Regression'] = np.round(lr.score(testX,testY),2)

print('Training Accuracy :',lr.score(trainX,trainY))
print('Testing Accuracy :',lr.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(lr.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])


print('\nCLASSIFICATION REPORT\n')
print(classification_report(lr.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sns.heatmap(con_mat, annot = True,fmt='d',cmap="YlGnBu")